# Load datasets from Hugging Face

Import datasets from Hugging Face Hub directly into Pixeltable tables.

**What's in this recipe:**
- Import Hugging Face datasets with one function call
- Automatic schema inference from dataset structure  
- Work with audio files in Pixeltable

## Problem

Hugging Face hosts thousands of multimodal datasets including audio. You need these datasets in Pixeltable to apply AI models, create embeddings, or run analysis.

## Solution

**What's in this recipe:**
- Import Hugging Face datasets with one function call
- Automatic schema inference from dataset structure
- Work with audio classification datasets in Pixeltable

You import Hugging Face datasets directly into Pixeltable tables using `pxt.create_table()` with the `source` parameter. This automatically infers the schema and loads the data, where you can immediately apply AI models and query results.

You can iterate on transformations before adding them to your table. Use `.select()` with `.collect()` to preview results on sample data—nothing is stored in your table. If you want to collect only the first few rows, use `.head(n)` instead of `.collect()`. Once you're satisfied, use `.add_computed_column()` to apply transformations to all rows in your table.

For more on this workflow, see [Get fast feedback on transformations](./dev-iterative-workflow.ipynb).

### Setup

In [ ]:
!uv add pixeltable datasets torchcodec twelvelabs

In [ ]:
import pixeltable as pxt

Load the [Drone Audio Detection Samples](https://huggingface.co/datasets/geronimobasso/drone-audio-detection-samples) dataset from Hugging Face, as described in the [Hugging Face documentation](https://huggingface.co/docs/datasets/en/package_reference/loading_methods).

In [ ]:
import datasets

drone_audio = datasets.load_dataset(
    'geronimobasso/drone-audio-detection-samples',
    split='train'
).cast_column('audio', datasets.Audio(decode=False))

### Create Pixeltable table

Now create a table and Pixeltable will map column types as needed.

In [ ]:
# Extract just the audio file paths
pxt.drop_dir('drone_audio', force=True)
pxt.create_dir('drone_audio')

In [ ]:
t = pxt.create_table(
    'drone_audio.samples',
    schema={
        'audio': pxt.Audio,
        'label': pxt.Int
    },
    comment='Drone audio detection samples with binary classification labels (drone vs non-drone) from Hugging Face'
)

In [ ]:
# Extract audio bytes
rows = []
for item in drone_audio:
    row = dict(item)
    row['audio'] = item['audio']['bytes']  # Use the bytes
    rows.append(row)

t.insert(rows)

In [ ]:
# View sample data
t.select(t.audio, t.label).head(5)

In [ ]:
# Check total count
t.count()

### Extract Audio Metadata

Add computed columns to extract metadata from the audio files.


In [ ]:
# Extract audio metadata
t.add_computed_column(metadata=t.audio.get_metadata())

In [ ]:
# Extract duration in seconds from metadata
t.add_computed_column(
    duration_seconds=t.metadata['streams'][0]['duration_seconds']
)

In [ ]:
# View the metadata
t.select(t.audio, t.label, t.duration_seconds).head(10)

### Publish to Pixeltable Cloud

Publish the table to make it available on Pixeltable Cloud.


In [ ]:
# Publish the table to Pixeltable Cloud
pxt.publish(
    'drone_audio.samples',
    'pxt://pixeltable:hugging-face/drone_audio',
    access='public'
)

### Add Audio Embeddings (Optional)

Add audio embeddings using TwelveLabs for audio similarity search.


In [ ]:
from pixeltable.functions import twelvelabs

In [ ]:
from getpass import getpass
import os

# Set TwelveLabs API key for audio embeddings
os.environ['TWELVELABS_API_KEY'] = getpass('TwelveLabs API Key:')

In [ ]:
# Add TwelveLabs audio embeddings for similarity search
# Pixeltable handles the embedding computation automatically
t.add_embedding_index(
    'audio',
    embedding=twelvelabs.embed.using(
        model_name='Marengo-retrieval-2.7'
    )
)

## See also

- [Drone Audio Detection Dataset on Hugging Face](https://huggingface.co/datasets/geronimobasso/drone-audio-detection-samples)
- [Working with Audio in Pixeltable](https://docs.pixeltable.com/docs/working-with-audio)
- [TwelveLabs Integration](https://docs.pixeltable.com/api/functions/twelvelabs)